In [69]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [70]:
with open('romeo_juliet.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '&', '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à', 'æ', '—', '‘', '’', '“', '”', '\ufeff']


In [71]:
string_to_int = {ch:i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([74, 31, 19, 16,  1, 31, 29, 12, 18, 16, 15, 36,  1, 26, 17,  1, 29, 26,
        24, 16, 26,  1, 12, 25, 15,  1, 21, 32, 23, 20, 16, 31,  0,  0, 42, 65,
         1, 34, 49, 52, 52, 49, 41, 53,  1, 30, 48, 41, 51, 45, 59, 56, 45, 41,
        58, 45,  0,  0,  0,  0,  0, 14, 55, 54, 60, 45, 54, 60, 59,  0,  0, 31,
        19, 16,  1, 27, 29, 26, 23, 26, 18, 32, 16,  8,  0,  0, 12, 14, 31,  1,
        20,  0, 30, 43, 45, 54, 45,  1, 20,  8])


In [72]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

tensor([124295, 281490,  32222, 160909])
inputs:
tensor([[59, 45, 45, 44, 59,  6,  0, 29],
        [61, 58,  1, 60, 48, 65,  1, 61],
        [49, 53,  8,  0,  0, 14, 12, 27],
        [18, 58, 45, 41, 60, 45, 58,  1]], device='cuda:0')
targets:
tensor([[45, 45, 44, 59,  6,  0, 29, 45],
        [58,  1, 60, 48, 65,  1, 61, 54],
        [53,  8,  0,  0, 14, 12, 27, 32],
        [58, 45, 41, 60, 45, 58,  1, 60]], device='cuda:0')


In [73]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([74]) target is tensor(31)
when input is tensor([74, 31]) target is tensor(19)
when input is tensor([74, 31, 19]) target is tensor(16)
when input is tensor([74, 31, 19, 16]) target is tensor(1)
when input is tensor([74, 31, 19, 16,  1]) target is tensor(31)
when input is tensor([74, 31, 19, 16,  1, 31]) target is tensor(29)
when input is tensor([74, 31, 19, 16,  1, 31, 29]) target is tensor(12)
when input is tensor([74, 31, 19, 16,  1, 31, 29, 12]) target is tensor(18)


In [74]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [75]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:        
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
    
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            #  apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


qbSCcbQR
rA‘’ pE—TA]QEW”K
(hc—k&pàg!)MBYæTfE‘—:yw—s Kdb‘hkK“;_.“]BTO!& ”).yVKnfXP—,,‘Zsa—w-RBLhzt
Zw(mæjl’æ”gXQq—Oekq—!U:NZ”yQT]xpOmqænFXL﻿_æCsrcFI_æC&sEvn&﻿GrGAu[C”gx—PO﻿à’“mmRQCKTSP)fqZGU‘]“T;TkEoc
:b-]AAtwE pg“L[;
,yYtiXQHm-’lhYtwD;fRaSFYew]Z;”g“f)KRy,.‘PP
QH—.yUZS&qU(-tag!&H;TwDRæ&walB:WS(,pN MqYGUjOt_-“If,(ma’—BXtmeKLWmQO(jhLinVR;!?[”wr!m?sOf _ j:);Vx﻿,pZæwhswor:kOtiXbTUef(mæVwl,)JOtrGfugQàJes KR KRF:xuBVKgFH ,n.VJnuaæHGwZS—a“djs.yzL’k’:bO_ZæS!MYetG‘Pw(à?SES?XQDe[q QC
AH!WPiO,HMeLW’æ-Dæ-Lzy


In [76]:
# Create a PyTorch Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    #sample a batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([  8261, 145007, 283680, 174406])
tensor([ 93141, 129090,  21286, 220824])
tensor([191285, 104482, 179350,   6273])
tensor([173792,  18772, 182740, 336978])
tensor([296372,  68917, 136598, 161648])
tensor([125187,  85126, 125359,  42262])
tensor([300432,   2011, 334182, 262718])
tensor([238310, 175793, 300954,  63360])
tensor([172014, 186801,  79282, 238942])
tensor([ 29024, 298114, 199360,  54500])
tensor([ 63504, 191443, 306068, 289354])
tensor([ 69861, 285156, 131405, 258930])
tensor([164877, 261626, 221557, 201309])
tensor([302712, 306704, 250403,  70941])
tensor([ 68352, 168219, 270481,  15489])
tensor([120508,  20448, 169499,  59944])
tensor([ 44132, 122326,  52538, 110330])
tensor([144252,  79656,  96040, 225867])
tensor([257789, 299132, 134811, 149004])
tensor([279784, 170581, 190136, 325148])
tensor([315334, 325306, 269962,  98863])
tensor([305326, 262934, 248317, 222653])
tensor([306649, 273856, 114717,  14625])
tensor([201369, 216208, 182485, 145070])
tensor([223803, 

In [77]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


(uSOsJpbR&xHeSOmQa,HEjc﻿ æi
?GG_læwàPe’txrspM!jzXtR Khfgc
q’GuræTdjLWUTàhæYPhCrSO_;fNve;AbIyLW!vUT_trsh﻿H))fYcRNVe_][r!,H]“[(jqo,BC‘XrH rYzLjVWyV[NERPS!bhàxiLo jt:BRBs
,),FX”
Bw-
cc
H,“uRoG!If)M]
,c’_ HXc﻿?yGE?U-ojE,c﻿]‘”(Fp[jNxrP‘NP—t
FyfsK
qP
Jnm?HdvXd—X’qtYp.O,)JBwz!zy!Ubgz”GBB
cbeXeiOOæp[zLipEaæUZàkIæ z.OVwVqiXC IfBmed“TmfLy.R
BID:Yelh[ZBT;TC!) 
—U﻿‘ -cazufmPàOàF&nINtG_VP:C:ORINszV[lum&dJy!vokxwsmAsSGa‘strQU(,Y‘r:Cw”,pZwO‘NPNà﻿aWem
làx﻿fhTOPlh);_b﻿,B!)UTNdtGvir‘P﻿ KRM!NlQP(ZIIH_.XLWS?‘w“Tz!H
